# Размещение баннеров

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив fsq.zip с этой страницы.

Нас будет интересовать файл checkins.dat. 

Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас.

In [1]:
import csv

with open('checkins.dat') as dat_file, open('checkins.csv', 'w') as csv_file:
    csv_writer = csv.writer(csv_file)

    for line in dat_file:
      row = [field.strip() for field in line.split('|')]
      csv_writer.writerow(row)

С помощью pandas построим DataFrame и убедимся, что все 396634 строки с координатами считаны успешно.

In [2]:
import pandas as pd

df = pd.read_csv('checkins.csv')
df.head

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<bound method NDFrame.head of                                                        id  ...           created_at
0       ---------+---------+----------+---------------...  ...                  NaN
1                                                  984301  ...  2012-04-21 17:39:01
2                                                  984222  ...  2012-04-21 17:43:47
3                                                  984315  ...  2012-04-21 17:37:18
4                                                  984234  ...  2012-04-21 17:43:43
...                                                   ...  ...                  ...
877607                                             195993  ...  2011-12-13 01:02:54
877608                                             196195  ...  2011-12-13 01:07:52
877609                                             196219  ...  2011-12-13 01:04:50
877610                                             196536  ...  2011-12-13 02:14:07
877611                                        

In [3]:
df.columns

Index(['id', 'user_id', 'venue_id', 'latitude', 'longitude', 'created_at'], dtype='object')

In [4]:
df_coord = df[['latitude', 'longitude']]

In [5]:
df = df_coord.dropna()

In [6]:
df

,latitude,longitude
2,38.895112,-77.036366
4,33.800745,-84.410520
8,45.523452,-122.676207
10,40.764462,-111.904565
11,33.448377,-112.074037
...,...,...
877596,26.142036,-81.794810
877598,40.633125,-89.398528
877603,37.714145,-122.250000
877605,41.878114,-87.629798


Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

Примечание:на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.


In [7]:
df_100 = df.head(100000)

In [8]:
df_100

,latitude,longitude
2,38.895112,-77.036366
4,33.800745,-84.410520
8,45.523452,-122.676207
10,40.764462,-111.904565
11,33.448377,-112.074037
...,...,...
233789,33.575000,-117.725556
233790,37.629349,-122.400087
233794,29.762884,-95.383061
233798,32.802955,-96.769923


In [9]:
from sklearn.cluster import MeanShift
X = df_100
clustering = MeanShift(bandwidth=0.1).fit(X)

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

In [10]:
clustering.labels_

array([ 5,  7, 30, ..., 25, 19,  4])

In [11]:
clustering.cluster_centers_

array([[  40.7177164 ,  -73.99183542],
       [  33.44943805, -112.00213969],
       [  33.44638027, -111.90188756],
       ...,
       [ -37.8229826 ,  145.1811902 ],
       [ -41.2924945 ,  174.7732353 ],
       [ -45.0311622 ,  168.6626435 ]])

In [12]:
max(clustering.labels_)

3230

In [13]:
from collections import Counter

clusts = Counter(clustering.labels_)

clusts_15 = {}

for k, i in clusts.items():
  if i > 15:
    clusts_15.update({k: i})

In [14]:
centers = list(clustering.cluster_centers_)

In [15]:
centers_15 = [list(centers[k]) for k in clusts_15.keys()]

In [16]:
def distance(point1, point2):
    return ((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)**0.5

In [17]:
offices = [[33.751277, -118.188740], 
           [25.867736, -80.324116], 
           [51.503016, -0.075479], 
           [52.378894, 4.885084],
           [39.366487, 117.036146], 
           [-33.868457, 151.205134]]

dist = []


for c in centers_15:
  dist_offices = []
  for of in offices:
    dist_offices.append(distance(c, of))
  dist.append(min(dist_offices))


In [19]:
centers_15[dist.index(min(dist))]

[-33.86063042857143, 151.20477592857145]